# 问卷调查反馈表分析

`studentsInfo.xlsx`文件中有`5`张表，保存了`50`名学生问卷抽样调查的结果。

## 研究问题

1. 男生、女生对`数据科学`课程的兴趣程序和成绩的变化趋势

2. 学生来自的省份及性别于成绩是否存在关系

3. 学生身高、体重达标状况

## 导入库

In [ ]:
import pandas as pd
import numpy as np

## 读取数据

In [ ]:
df1 = pd.read_excel('studentsInfo.xlsx', sheet_name='Group1', index_col=0)
df1

由于表中的第`0`列已经有`序号`了，这里`index_col=0`表示将第`0`列作为行索引。

继续读取剩下的表：

In [ ]:
df2 = pd.read_excel('studentsInfo.xlsx', sheet_name='Group2', index_col=0)
df3 = pd.read_excel('studentsInfo.xlsx', sheet_name='Group3', index_col=0)
df4 = pd.read_excel('studentsInfo.xlsx', sheet_name='Group4', index_col=0)
df5 = pd.read_excel('studentsInfo.xlsx', sheet_name='Group5', index_col=0)

合并成一张表：

In [ ]:
stu = pd.concat([df1, df2, df3, df4, df5], axis=0)
stu

读取多张表可以写成循环：

In [ ]:
df_all = [
    pd.read_excel('studentsInfo.xlsx', sheet_name=f'Group{i}', index_col=0)
    for i in range(1, 6)
]

stu = pd.concat(df_all, axis=0)
stu

In [ ]:
stu.shape

## 数据清洗

去除重复数据：

In [ ]:
stu.drop_duplicates(inplace=True)
stu.shape

去除缺失值过多（$ \ge 2 $）的行：

In [ ]:
stu.dropna(thresh=8, inplace=True)
stu.shape

检测数据中是否存在`NaN`：

In [ ]:
stu.isnull().any()

`年龄`和`成绩`列存在缺失值。

`成绩`可以按照平均分填充。

由于接受调查的都为大二学生，所以`年龄`可以都使用`20`填充。

In [ ]:
stu.fillna({
    '成绩': stu['成绩'].mean(),
    '年龄': 20
}, inplace=True)

stu.isnull().any()

数据清洗完成，此时已经没有缺失值了。

### 数据分析

#### 分析成绩与课程兴趣的关系

将学生按照`成绩`排序，统计成绩`优秀（大于或等于90分）`和`不及格（小于60分）`的学生**个数**。

In [ ]:
stu_grade = stu.sort_values(by='成绩', ascending=False)
stu_grade

In [ ]:
excellent = (stu_grade['成绩'] >= 90).sum()
excellent

In [ ]:
fail = (stu_grade['成绩'] < 60).sum()
fail

分别计算成绩`优秀`与`不及格`的学生`平均课程兴趣`程度。

In [ ]:
excellent_mean = stu_grade[:excellent]['课程兴趣'].mean()
excellent_mean

In [ ]:
fail_mean = stu_grade[-fail:]['课程兴趣'].mean()
fail_mean

计算**全体**学生成绩的`平均分`与`平均课程兴趣`。

In [ ]:
total_mean = stu_grade[['成绩', '课程兴趣']].mean()
total_mean

查看`成绩`与`课程兴趣`是否有相关性：

In [ ]:
stu_grade['成绩'].corr(stu_grade['课程兴趣'])

#### 分析性别、省份与成绩的相关性

分析`性别`、`省份`与`成绩`是否存在相关性，由于`性别`和`省份`数据均为字符型数据，无法用`corr()`来计算，可以通过分组计算均值。

先根据`性别`进行分组，可以发现男、女学生各`24`名。

In [ ]:
sex_grouped = stu.groupby(['性别'])
sex_counts = sex_grouped.count()
sex_counts

计算男、女学生的`平均成绩`：

In [ ]:
sex_mean = sex_grouped.aggregate({'成绩': np.mean})
sex_mean

可以发现，男生平均成绩比女生略高。

同样对`省份`进行分组：

In [ ]:
province_grouped = stu.groupby(['省份'])
province_counts = province_grouped.count()
province_counts

计算不同`省份`学生的`平均成绩`：

In [ ]:
province_mean = province_grouped.aggregate({'成绩': np.mean})
province_mean

通过观察可以轻松发现，哪些省份的平均成绩更高。

但是，由于分组的样本太少（每个省份只有1~3名学生），分析结果不具备统计学参考价值。

#### 统计身高、体重达标状况

计算学生的`BMI`：

$$
BMI = {\text{Weight (kg)} \over \text{Height (m)}^2}
$$

中国BMI标准：

| 分类 | 范围 |
| --- | --- |
| 偏轻 | $ < 18.5 $ |
| 正常 | $ 18.5 \sim 23.9 $ |
| 偏胖 | $ 24 \sim 27.9 $ |
| 肥胖 | $ > 28 $ |

In [ ]:
stu['BMI'] = stu['体重'] / np.square(stu['身高'] / 100)
stu

找出`BMI`四分位数：

In [ ]:
stu['BMI'].quantile([.25, .5, .75])

统计`BMI >= 28`的人数：

In [ ]:
(stu['BMI'] >= 28).sum()